# Reverse-engineering of the surron ultrabee can bus

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json

---
## Reading data from Files

In [26]:
filesToRead = ["20240427_CAN_Trace_Motorrad_T000.asc", "20240427_CAN_Trace_Motorrad_T001.asc"]

dataPointsAsc = []
for fileName in filesToRead:
    with open(fileName) as f:
        content = f.readlines()
        for line in content:
            dataPointsAsc.append({
                "lineId": len(dataPointsAsc),
                "Id": line[101:len(line) - 1],
                "dlc": line[38:39],
                "msg": line[40:64],
                "time": line[:12],
            })

tmpCleanedArray = []
for e in dataPointsAsc:
    if (e["Id"] != '') & (e["Id"] != ' ') & (e["dlc"] != '') & (e["dlc"] != ' '):
        tmpCleanedArray.append(e)

dataPointsAsc = tmpCleanedArray

for pos in range(len(dataPointsAsc)):
    dataPointsAsc[pos]["lineId"] = pos

messagesPerId = []

# print(set(list(map(lambda row: (row["Id"]), dataPointsAsc))))

for singleId in set(list(map(lambda row: (row["Id"]), dataPointsAsc))):
    var = list(map(lambda row: (row["msg"]), filter(lambda single: single["Id"] == singleId, dataPointsAsc)))
    messagesPerId.append({
        "Id": singleId,
        "cnt": len(var),
        "msg": var
    })

df = pd.DataFrame(messagesPerId)
df

,Id,cnt,msg
0,785,1327,"[02 00 00 00 29 00 CB 02 , 02 00 00 00 28 00 C..."
1,786,52,"[01 32 30 32 33 30 33 32 , 02 35 30 30 30 31 0..."
2,789,667,"[05 05 03 03 01 00 03 03 , 05 05 03 03 01 00 0..."
3,772,672,"[02 54 51 4C 30 30 31 47 , 03 44 35 4E 30 30 3..."
4,690,1,[00 08 02 00 00 00 00 00 ]
5,801,1,[C1 00 FF FF FF FF FF FF ]
6,274,1328,"[00 00 3C 37 00 00 00 00 , 00 00 3C 37 00 00 0..."
7,787,27,"[44 30 38 41 31 31 30 36 , 44 30 38 41 31 31 3..."
8,513,672,"[00 00 C8 00 00 00 07 CF , 00 00 C8 00 00 00 0..."
9,417333259x,271,"[00 00 00 00 00 00 01 00 , 00 00 00 00 00 00 0..."


---
## Plot all DataPoints over time

In [ ]:
for message in messagesPerId:
    for i in range(8):
        searchedId = message["Id"]
        dataByte = i

        plotData = map(lambda row: ({"time": row["time"], "val": int(row["msg"].split()[dataByte], 16)}),
                       list(filter(lambda item: item['Id'] == searchedId, dataPointsAsc)))

        df = pd.DataFrame(plotData).plot("time", "val")
        df.set_title({"Id": searchedId, "dataPos": i})

# df = pd.DataFrame(plotData)
# df